In [1]:
import pandas as pd
import numpy as np

import xgboost as xgb
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix 

In [2]:
train = pd.read_csv('../organized_data/train_cleaned_v1.csv')
test = pd.read_csv('../organized_data/test_organized.csv')

In [6]:
for col in train.columns:
    print('{}\n'.format(str(col)))
    print(train[col].unique())
    print('\n')

individualnumber

[94230288  4684087 88026681 ... 98445787 13781030 96703898]


category_number

[9000 9030 9001 9046 9044 9007 9006 9022 9018 9038 9058 9049 9042 9029
 9036 9024 9032 9056 9043 9010 9002 9059 9004 9037 9055 9019 9031 9035
 9012 9054 9009 9040 9027 9021 9048 9017 9052 9039 9003 9005 9060 9015
 9008 9041 9020 9053 9061 9028 9011]


hakkedis_amt

[ 21.  17.  22.  38.  85.  27.  19.  33.  60.  43.  16.  23.  56.   9.
  24.  26.  32.  18.  39.  66.  57.  55.  45.  49.  15.  40.  28.  34.
  99.  20.  36.  37. 100. 121.  30.  35.  48.  41.  31.  78.   7.  44.
  14. 104.  73.  54.  90.  51.  52.  11.  63.   8.  79.  72.  29.  42.
  80.  77.  12.  71.  10. 125.  92.  25.  68.  69.  13. 136.  47.  65.
  62.  46.  70.  96.  67.  64.  50.  97. 118.  59. 112. 105.  83.  76.
  58.  53.  91.  75.  74. 109.  84.  93.  95. 123. 135. 116. 126. 113.
  94. 129. 131. 107. 180. 114. 111. 119. 122. 117. 115. 102.  87. 124.
  61. 120.  81.  89. 106. 103. 128. 108. 110.]


odul_amt

[ 2.  1.  

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13034 entries, 0 to 13033
Data columns (total 34 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   individualnumber                           13034 non-null  int64  
 1   category_number                            13034 non-null  int64  
 2   hakkedis_amt                               13034 non-null  float64
 3   odul_amt                                   13034 non-null  float64
 4   response                                   13034 non-null  int64  
 5   total_money_spent                          13034 non-null  float64
 6   total_discount                             13034 non-null  float64
 7   sanal_percent                              13034 non-null  float64
 8   shop_count                                 13034 non-null  float64
 9   months_since_last_shopping                 13034 non-null  float64
 10  days_shopped          

In [5]:
avg_age = round(train.age.mean())
train.age.iloc[train.query('age.isnull()').index] = avg_age
train.age.iloc[train.query('age<0').index] = avg_age

c:\Users\tree\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
c:\Users\tree\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [7]:
X = train.drop(['individualnumber', 'response', 'city_code', 'is_large_city'], axis=1).copy()
X.head()

,category_number,hakkedis_amt,odul_amt,total_money_spent,total_discount,sanal_percent,shop_count,months_since_last_shopping,days_shopped,gender,...,level2_relevant_category_volume_per_day,level3_relevant_category_volume_per_day,level4_relevant_category_volume_per_day,level1_relevant_category_quantity_per_day,level2_relevant_category_quantity_per_day,level3_relevant_category_quantity_per_day,level4_relevant_category_quantity_per_day,total_money_spent_per_day,discount_per_day,odul/hakkedis
0,9000,21.0,2.0,7118.94741,177.489978,0.015625,64.0,1.0,357.0,E,...,1.365154,0.675378,0.675378,0.270924,0.131653,0.100840,0.100840,19.941029,0.497171,0.095238
1,9000,17.0,1.0,10192.64563,31907.260487,0.000000,30.0,1.0,309.0,E,...,0.342718,0.342718,0.342718,0.113528,0.038835,0.038835,0.038835,32.985908,103.259743,0.058824
2,9030,22.0,2.0,11559.66676,231.976316,0.950000,40.0,1.0,350.0,K,...,0.396143,0.396143,0.396143,0.040000,0.022857,0.022857,0.022857,33.027619,0.662789,0.090909
3,9001,38.0,3.0,461.77380,1.738800,0.000000,4.0,3.0,188.0,E,...,0.202128,0.202128,0.202128,0.058511,0.010638,0.010638,0.010638,2.456244,0.009249,0.078947
4,9046,85.0,8.0,180.57000,19.349210,0.000000,6.0,1.0,94.0,K,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.920957,0.205843,0.094118


In [8]:
y = train['response'].copy()
y.unique()

array([0, 1], dtype=int64)

In [9]:
X_encoded = pd.get_dummies(X, columns=['gender', 'category_number'])
X_encoded.head()

,hakkedis_amt,odul_amt,total_money_spent,total_discount,sanal_percent,shop_count,months_since_last_shopping,days_shopped,age,level1_relevant_category_volume,...,category_number_9049,category_number_9052,category_number_9053,category_number_9054,category_number_9055,category_number_9056,category_number_9058,category_number_9059,category_number_9060,category_number_9061
0,21.0,2.0,7118.94741,177.489978,0.015625,64.0,1.0,357.0,39.0,996.749,...,0,0,0,0,0,0,0,0,0,0
1,17.0,1.0,10192.64563,31907.260487,0.000000,30.0,1.0,309.0,71.0,461.227,...,0,0,0,0,0,0,0,0,0,0
2,22.0,2.0,11559.66676,231.976316,0.950000,40.0,1.0,350.0,61.0,308.150,...,0,0,0,0,0,0,0,0,0,0
3,38.0,3.0,461.77380,1.738800,0.000000,4.0,3.0,188.0,38.0,115.700,...,0,0,0,0,0,0,0,0,0,0
4,85.0,8.0,180.57000,19.349210,0.000000,6.0,1.0,94.0,23.0,0.000,...,0,0,0,0,0,0,0,0,0,0


In [13]:
X_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13034 entries, 0 to 13033
Data columns (total 79 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   hakkedis_amt                               13034 non-null  float64
 1   odul_amt                                   13034 non-null  float64
 2   total_money_spent                          13034 non-null  float64
 3   total_discount                             13034 non-null  float64
 4   sanal_percent                              13034 non-null  float64
 5   shop_count                                 13034 non-null  float64
 6   months_since_last_shopping                 13034 non-null  float64
 7   days_shopped                               13034 non-null  float64
 8   age                                        13034 non-null  float64
 9   level1_relevant_category_volume            13034 non-null  float64
 10  level2_relevant_catego

In [131]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, stratify=y)
print('train ratio: {}\ntest ratio:{}'.format(sum(y_train)/len(y_train),sum(y_test)/len(y_test)))
print('#0/#1 is {}'.format((len(y)-sum(y))/sum(y)))

train ratio: 0.015447570332480818
test ratio:0.015342129487572876
#0/#1 is 63.84577114427861


In [170]:
xgb0 = xgb.XGBClassifier(scale_pos_weight=63, max_depth=8,learning_rate=0.13, n_estimators=50, objective='binary:logistic',
                         booster='gbtree', subsample=0.2, eval_metric='auc', colsample_bynode=.2)

eval_set = [(X_test,y_test)]
xgb0.fit(X_train, y_train, verbose=2, eval_set=eval_set)

[0]	validation_0-auc:0.82700
[2]	validation_0-auc:0.81306
[4]	validation_0-auc:0.79668
[6]	validation_0-auc:0.78057
[8]	validation_0-auc:0.82127
[10]	validation_0-auc:0.83301
[12]	validation_0-auc:0.83479
[14]	validation_0-auc:0.85216
[16]	validation_0-auc:0.84282
[18]	validation_0-auc:0.85068
[20]	validation_0-auc:0.85641
[22]	validation_0-auc:0.85111
[24]	validation_0-auc:0.85424
[26]	validation_0-auc:0.85081
[28]	validation_0-auc:0.84496
[30]	validation_0-auc:0.83864
[32]	validation_0-auc:0.83784
[34]	validation_0-auc:0.84524
[36]	validation_0-auc:0.84742
[38]	validation_0-auc:0.84016
[40]	validation_0-auc:0.83210
[42]	validation_0-auc:0.82904
[44]	validation_0-auc:0.82916
[46]	validation_0-auc:0.82581
[48]	validation_0-auc:0.83422
[49]	validation_0-auc:0.83333


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=0.2, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='auc', feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.13, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=8, max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=50, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [171]:
y_predict_test = xgb0.predict(X_test)
y_predict_train = xgb0.predict(X_train)
print('Test\nConfusion Matrix:')
print(confusion_matrix(y_test, y_predict_test))
print(classification_report(y_test,y_predict_test))
print(metrics.fbeta_score(y_test, y_predict_test, beta=0.5))
print('Train\nConfusion Matrix:')
print(confusion_matrix(y_train, y_predict_train))
print(classification_report(y_train,y_predict_train))
print(metrics.fbeta_score(y_train, y_predict_train, beta=0.5))

Test
Confusion Matrix:
[[3141   68]
 [  32   18]]
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      3209
           1       0.21      0.36      0.26        50

    accuracy                           0.97      3259
   macro avg       0.60      0.67      0.62      3259
weighted avg       0.98      0.97      0.97      3259

0.22842639593908634
Train
Confusion Matrix:
[[9454  170]
 [  33  118]]
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      9624
           1       0.41      0.78      0.54       151

    accuracy                           0.98      9775
   macro avg       0.70      0.88      0.76      9775
weighted avg       0.99      0.98      0.98      9775

0.45280122793553335


In [77]:
#MODEL 1
model1 = xgb.XGBClassifier(scale_pos_weight=64, max_depth=5,learning_rate=0.1, n_estimators=50, objective='binary:logistic',
                         booster='gbtree', subsample=0.5, eval_metric='auc', colsample_bynode=.2)

eval_set = [(X_test,y_test)]
model1.fit(X_train, y_train, verbose=2, eval_set=eval_set)

[0]	validation_0-auc:0.87190
[2]	validation_0-auc:0.88772
[4]	validation_0-auc:0.87957
[6]	validation_0-auc:0.88455
[8]	validation_0-auc:0.87817
[10]	validation_0-auc:0.86512
[12]	validation_0-auc:0.86309
[14]	validation_0-auc:0.86722
[16]	validation_0-auc:0.87765
[18]	validation_0-auc:0.88547
[20]	validation_0-auc:0.88622
[22]	validation_0-auc:0.89099
[24]	validation_0-auc:0.89236
[26]	validation_0-auc:0.89401
[28]	validation_0-auc:0.89829
[30]	validation_0-auc:0.89678
[32]	validation_0-auc:0.89743
[34]	validation_0-auc:0.89946
[36]	validation_0-auc:0.89892
[38]	validation_0-auc:0.90025
[40]	validation_0-auc:0.89872
[42]	validation_0-auc:0.90203
[44]	validation_0-auc:0.90039
[46]	validation_0-auc:0.89888
[48]	validation_0-auc:0.89768
[49]	validation_0-auc:0.89898


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=0.2, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='auc', feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=5, max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=50, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [78]:
#MODEL 2
model2 = xgb.XGBClassifier(scale_pos_weight=63, max_depth=6,learning_rate=0.1, n_estimators=50, objective='binary:logistic',
                         booster='gbtree', subsample=0.2, eval_metric='auc', colsample_bynode=.2)

eval_set = [(X_test,y_test)]
model2.fit(X_train, y_train, verbose=2, eval_set=eval_set)

[0]	validation_0-auc:0.84039
[2]	validation_0-auc:0.86620
[4]	validation_0-auc:0.87479
[6]	validation_0-auc:0.86580
[8]	validation_0-auc:0.85529
[10]	validation_0-auc:0.85919
[12]	validation_0-auc:0.88963
[14]	validation_0-auc:0.88545
[16]	validation_0-auc:0.88386
[18]	validation_0-auc:0.88485
[20]	validation_0-auc:0.88719
[22]	validation_0-auc:0.89581
[24]	validation_0-auc:0.89564
[26]	validation_0-auc:0.89264
[28]	validation_0-auc:0.89351
[30]	validation_0-auc:0.89126
[32]	validation_0-auc:0.89508
[34]	validation_0-auc:0.89260
[36]	validation_0-auc:0.89415
[38]	validation_0-auc:0.90023
[40]	validation_0-auc:0.89860
[42]	validation_0-auc:0.89773
[44]	validation_0-auc:0.89744
[46]	validation_0-auc:0.89192
[48]	validation_0-auc:0.88945
[49]	validation_0-auc:0.88934


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=0.2, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='auc', feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=50, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [79]:
#MODEL 3
model3 = xgb.XGBClassifier(scale_pos_weight=63, max_depth=7,learning_rate=0.1, n_estimators=50, objective='binary:logistic',
                         booster='gbtree', subsample=0.2, eval_metric='auc', colsample_bynode=.25)

eval_set = [(X_test,y_test)]
model3.fit(X_train, y_train, verbose=2, eval_set=eval_set)

[0]	validation_0-auc:0.83385
[2]	validation_0-auc:0.86157
[4]	validation_0-auc:0.87022
[6]	validation_0-auc:0.85066
[8]	validation_0-auc:0.85541
[10]	validation_0-auc:0.87001
[12]	validation_0-auc:0.84900
[14]	validation_0-auc:0.86250
[16]	validation_0-auc:0.86155
[18]	validation_0-auc:0.87964
[20]	validation_0-auc:0.87912
[22]	validation_0-auc:0.88351
[24]	validation_0-auc:0.88441
[26]	validation_0-auc:0.88727
[28]	validation_0-auc:0.88629
[30]	validation_0-auc:0.88639
[32]	validation_0-auc:0.88830
[34]	validation_0-auc:0.89018
[36]	validation_0-auc:0.88781
[38]	validation_0-auc:0.88992
[40]	validation_0-auc:0.89121
[42]	validation_0-auc:0.88845
[44]	validation_0-auc:0.88710
[46]	validation_0-auc:0.88207
[48]	validation_0-auc:0.88609
[49]	validation_0-auc:0.88613


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=0.25, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='auc', feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=7, max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=50, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [ ]:
#MODEL 4
model4 = xgb.XGBClassifier(scale_pos_weight=63, max_depth=8,learning_rate=0.08, n_estimators=50, objective='binary:logistic',
                         booster='gbtree', subsample=0.2, eval_metric='auc', colsample_bynode=.2)

eval_set = [(X_test,y_test)]
model4.fit(X_train, y_train, verbose=2, eval_set=eval_set)

In [ ]:
#MODEL 5
model5 = xgb.XGBClassifier(scale_pos_weight=63, max_depth=8,learning_rate=0.13, n_estimators=50, objective='binary:logistic',
                         booster='gbtree', subsample=0.2, eval_metric='auc', colsample_bynode=.2)

eval_set = [(X_test,y_test)]
model5.fit(X_train, y_train, verbose=2, eval_set=eval_set)

PREDICTION AND SUBMISSION OF REAL TEST DATA

In [80]:
avg_age = round(test.age.mean())
test.age.iloc[test.query('age.isnull()').index] = avg_age
test.age.iloc[test.query('age<0').index] = avg_age
X_TEST = test.drop(['individualnumber', 'city_code', 'is_large_city'], axis=1).copy()
X_TEST_encoded = pd.get_dummies(X_TEST, columns=['gender', 'category_number'])

c:\Users\tree\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
c:\Users\tree\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [172]:
y_predict_xgb0 = xgb0.predict(X_TEST_encoded)
submission = pd.DataFrame()
submission['individualnumber'] = test['individualnumber']
submission['response'] = y_predict_xgb0
submission.to_csv('./submissions/submission4.csv', index=False)

PARAMETER TUNING

In [173]:
from skopt import BayesSearchCV
from sklearn.metrics import f1_score

In [174]:
train = pd.read_csv('../organized_data/train_cleaned_v1.csv')
test = pd.read_csv('../organized_data/test_organized.csv')
avg_age = round(train.age.mean())
train.age.iloc[train.query('age.isnull()').index] = avg_age
train.age.iloc[train.query('age<0').index] = avg_age
X = train.drop(['individualnumber', 'response', 'city_code', 'is_large_city'], axis=1).copy()
y = train['response'].copy()
X_encoded = pd.get_dummies(X, columns=['gender', 'category_number'])

c:\Users\tree\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
c:\Users\tree\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [175]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, stratify=y)
print('train ratio: {}\ntest ratio:{}'.format(sum(y_train)/len(y_train),sum(y_test)/len(y_test)))
print('#0/#1 is {}'.format((len(y)-sum(y))/sum(y)))

train ratio: 0.015447570332480818
test ratio:0.015342129487572876
#0/#1 is 63.84577114427861


In [203]:
#TUNING ROUND 1
param_grid = {'gamma':[0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4],
              'learning_rate':[0.075, 0.1, 0.125],
              'max_depth':[9, 10, 11, 12],
              'n_estimators':[90, 95, 100, 105, 110, 120, 150, 180],
              'subsample':[0.175, 0.2, 0.225],
              'colsample_bynode':[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
              'colsample_bytree':[0.6, 0.7, 0.8, 0.9],
              'reg_alpha':[0, 0.1, 0.2, 0.4, 0.8],
              'reg_lambda':[0, 0.1, 0.2, 0.4, 0.8]}

gcvj = np.cumsum([len(x) for x in param_grid.values()])[-1]
bcvj = int(gcvj)

xgbc = xgb.XGBClassifier(objective='binary:logistic',
                          booster='gbtree',
                          scale_pos_weight=63,
                          eval_metric='auc')

clf = BayesSearchCV(estimator=xgbc, search_spaces=param_grid, n_iter=bcvj, scoring='f1', cv=3, return_train_score=True, verbose=3)
clf.fit(X_train, y_train)

df = pd.DataFrame(clf.cv_results_)

train_pred = clf.predict(X_train)
test_pred = clf.predict(X_test)

print('Test\n')
print(confusion_matrix(y_test, test_pred))
print(classification_report(y_test,test_pred))
print(metrics.fbeta_score(y_test, test_pred, beta=0.5))
print('Train\n')
print(confusion_matrix(y_train, train_pred))
print(classification_report(y_train,train_pred))
print(metrics.fbeta_score(y_train, train_pred, beta=0.5))

bp = clf.best_params_

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END colsample_bynode=0.4, colsample_bytree=0.4, gamma=0.1, learning_rate=0.175, max_depth=9, n_estimators=80, subsample=0.175;, score=(train=0.725, test=0.200) total time=   1.1s
[CV 2/3] END colsample_bynode=0.4, colsample_bytree=0.4, gamma=0.1, learning_rate=0.175, max_depth=9, n_estimators=80, subsample=0.175;, score=(train=0.803, test=0.232) total time=   1.1s
[CV 3/3] END colsample_bynode=0.4, colsample_bytree=0.4, gamma=0.1, learning_rate=0.175, max_depth=9, n_estimators=80, subsample=0.175;, score=(train=0.800, test=0.194) total time=   1.2s
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END colsample_bynode=0.1, colsample_bytree=0.2, gamma=25.6, learning_rate=0.175, max_depth=7, n_estimators=60, subsample=0.225;, score=(train=0.183, test=0.145) total time=   0.3s
[CV 2/3] END colsample_bynode=0.1, colsample_bytree=0.2, gamma=25.6, learning_rate=0.175, max_depth=7, n_estimators=60, subsample

In [204]:
sorted_df = df.sort_values(by='rank_test_score')
sorted_df.to_csv('stats_round1.csv',index=False)
bp

OrderedDict([('colsample_bynode', 0.4),
             ('colsample_bytree', 0.4),
             ('gamma', 12.8),
             ('learning_rate', 0.075),
             ('max_depth', 7),
             ('n_estimators', 90),
             ('subsample', 0.175)])

In [205]:
#TUNING ROUND 2
param_grid = {'gamma':[6.4, 12.8, 25.6, 51.2],
              'learning_rate':[0.05, 0.075, 0.1],
              'max_depth':[5, 6, 7, 8],
              'n_estimators':[60, 70, 80, 90, 100],
              'subsample':[0.175, 0.2, 0.225],
              'colsample_bynode':[0.4, 0.5, 0.6, 0.7, 0.8],
              'colsample_bytree':[0.2, 0.3, 0.4],
              'reg_alpha':[0, 0.1, 0.2, 0.4, 0.8],
              'reg_lambda':[0, 0.1, 0.2, 0.4, 0.8]}

gcvj = np.cumsum([len(x) for x in param_grid.values()])[-1]
bcvj = int(gcvj)

xgbc = xgb.XGBClassifier(objective='binary:logistic',
                          booster='gbtree',
                          scale_pos_weight=63,
                          eval_metric='auc')

clf = BayesSearchCV(estimator=xgbc, search_spaces=param_grid, n_iter=bcvj, scoring='f1', cv=3, return_train_score=True, verbose=3)
clf.fit(X_train, y_train)

df2 = pd.DataFrame(clf.cv_results_)

train_pred = clf.predict(X_train)
test_pred = clf.predict(X_test)

print('Test\n')
print(confusion_matrix(y_test, test_pred))
print(classification_report(y_test,test_pred))
print(metrics.fbeta_score(y_test, test_pred, beta=0.5))
print('Train\n')
print(confusion_matrix(y_train, train_pred))
print(classification_report(y_train,train_pred))
print(metrics.fbeta_score(y_train, train_pred, beta=0.5))

bp2 = clf.best_params_

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END colsample_bynode=0.7, colsample_bytree=0.4, gamma=12.8, learning_rate=0.075, max_depth=6, n_estimators=70, reg_alpha=0.0, reg_lambda=0.1, subsample=0.2;, score=(train=0.505, test=0.199) total time=   0.9s
[CV 2/3] END colsample_bynode=0.7, colsample_bytree=0.4, gamma=12.8, learning_rate=0.075, max_depth=6, n_estimators=70, reg_alpha=0.0, reg_lambda=0.1, subsample=0.2;, score=(train=0.478, test=0.279) total time=   0.7s
[CV 3/3] END colsample_bynode=0.7, colsample_bytree=0.4, gamma=12.8, learning_rate=0.075, max_depth=6, n_estimators=70, reg_alpha=0.0, reg_lambda=0.1, subsample=0.2;, score=(train=0.414, test=0.309) total time=   0.7s
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END colsample_bynode=0.7, colsample_bytree=0.3, gamma=25.6, learning_rate=0.1, max_depth=6, n_estimators=70, reg_alpha=0.0, reg_lambda=0.2, subsample=0.175;, score=(train=0.475, test=0.209) total time=   0.7s
[CV 2/3] E

In [207]:
sorted_df2 = df2.sort_values(by='rank_test_score')
sorted_df2.to_csv('stats_round2.csv',index=False)
bp2

OrderedDict([('colsample_bynode', 0.7),
             ('colsample_bytree', 0.4),
             ('gamma', 25.6),
             ('learning_rate', 0.05),
             ('max_depth', 8),
             ('n_estimators', 90),
             ('reg_alpha', 0.2),
             ('reg_lambda', 0.0),
             ('subsample', 0.175)])

In [277]:
clf_xgb = xgb.XGBClassifier(scale_pos_weight=63, max_depth=8,learning_rate=0.1, n_estimators=90, objective='binary:logistic',
                         booster='gbtree', subsample=0.175, eval_metric='auc', colsample_bynode=.5,colsample_bytree=.4,
                         reg_alpha=0.2, reg_lambda=0.0, gamma=20)

eval_set = [(X_test,y_test)]
clf_xgb.fit(X_train, y_train, verbose=2, eval_set=eval_set)

[0]	validation_0-auc:0.70764
[2]	validation_0-auc:0.72812
[4]	validation_0-auc:0.71587
[6]	validation_0-auc:0.74456
[8]	validation_0-auc:0.81324
[10]	validation_0-auc:0.81792
[12]	validation_0-auc:0.81677
[14]	validation_0-auc:0.81264
[16]	validation_0-auc:0.83711
[18]	validation_0-auc:0.83040
[20]	validation_0-auc:0.83349
[22]	validation_0-auc:0.84161
[24]	validation_0-auc:0.83646
[26]	validation_0-auc:0.84944
[28]	validation_0-auc:0.84461
[30]	validation_0-auc:0.84551
[32]	validation_0-auc:0.86557
[34]	validation_0-auc:0.86633
[36]	validation_0-auc:0.87204
[38]	validation_0-auc:0.87726
[40]	validation_0-auc:0.88039
[42]	validation_0-auc:0.87864
[44]	validation_0-auc:0.88149
[46]	validation_0-auc:0.88559
[48]	validation_0-auc:0.88522
[50]	validation_0-auc:0.88274
[52]	validation_0-auc:0.88164
[54]	validation_0-auc:0.88659
[56]	validation_0-auc:0.88735
[58]	validation_0-auc:0.88842
[60]	validation_0-auc:0.89091
[62]	validation_0-auc:0.89078
[64]	validation_0-auc:0.88719
[66]	validation

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=0.5, colsample_bytree=0.4,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='auc', feature_types=None, gamma=20, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=8, max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=90, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [279]:
train_pred = clf_xgb.predict(X_train)
test_pred = clf_xgb.predict(X_test)

print('Test\n')
print(confusion_matrix(y_test, test_pred))
print(classification_report(y_test,test_pred))
print(metrics.fbeta_score(y_test, test_pred, beta=0.5))
print('Train\n')
print(confusion_matrix(y_train, train_pred))
print(classification_report(y_train,train_pred))
print(metrics.fbeta_score(y_train, train_pred, beta=0.5))

Test

[[3146   63]
 [  37   13]]
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      3209
           1       0.17      0.26      0.21        50

    accuracy                           0.97      3259
   macro avg       0.58      0.62      0.60      3259
weighted avg       0.98      0.97      0.97      3259

0.18361581920903955
Train

[[9464  160]
 [  31  120]]
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      9624
           1       0.43      0.79      0.56       151

    accuracy                           0.98      9775
   macro avg       0.71      0.89      0.77      9775
weighted avg       0.99      0.98      0.98      9775

0.47206923682140045
